In [18]:
import praw # Python Reddit API Wrapper

# Create a Reddit instance
reddit = praw.Reddit(
    client_id='51f3IBDpHGRzaGh0FGj0ng',
    client_secret=	'C-mvx1K5Flj_4kM-Grc9FLT2_bHxag',
    user_agent='TradingAnalysisApp by /u/Ok_Ability_9624'
)


In [19]:
print(reddit.read_only)

True


In [23]:
# Fetch and print the titles of the top 10 hot posts in the 'test' subreddit
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)

ResponseException: received 401 HTTP response